### Importation packages

In [1]:
import numpy as np
import pandas as pd
import re

### Jointures tables (à ne pas faire retourner)

In [2]:
annees = ['2019']
compet = ['cross', 'stade', 'salle']
data_tot = pd.DataFrame()

for an in annees: 
    for comp in compet: 
        data = pd.read_excel("C:/Users/alexa/OneDrive/Documents/Projets/Données athlé/Donnees/Donnees competitions/Resultats_" + comp + '_' + an + ".xlsx")
        data['Annee_competition']=an
        data['Type_competition']=comp
        data_tot = pd.concat([data_tot, data])    

In [3]:
annees = ['2020', '2021', '2022']
compet = ['cross', 'HS', 'salle', 'stade']

for an in annees: 
    for comp in compet: 
        data = pd.read_excel("C:/Users/alexa/OneDrive/Documents/Projets/Données athlé/Donnees/Donnees competitions/Resultats_" + comp + '_' + an + ".xlsx")
        data['Annee_competition']=an
        data['Type_competition']=comp
        data_tot = pd.concat([data_tot, data]) 



In [4]:
annees = ['2023']
compet = ['cross', 'salle']

for an in annees: 
    for comp in compet: 
        data = pd.read_excel("C:/Users/alexa/OneDrive/Documents/Projets/Données athlé/Donnees/Donnees competitions/Resultats_" + comp + '_' + an + ".xlsx")
        data['Annee_competition']=an
        data['Type_competition']=comp
        data_tot = pd.concat([data_tot, data])     

In [5]:
data_tot.to_csv(f'Resultats_totaux.csv') 

#### Importation données

In [2]:
data_tot = pd.read_csv("C:/Users/alexa/OneDrive/Documents/Projets/Resultats_totaux.csv")
data_tot.head()

C:\Users\alexa\AppData\Local\Temp\ipykernel_81500\2784364973.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  data_tot = pd.read_csv("C:/Users/alexa/OneDrive/Documents/Projets/Resultats_totaux.csv")


,Unnamed: 0.1,Unnamed: 0,Position,Nom_athlete,Performance,Annee_naissance,Categorie,Club,Département,Ligue,Epreuve,Nom_competition,Lieu_competition,Date_competition,Heure_epreuve,Departement_competition,Region_competition,Label_competition,Annee_competition,Type_competition
0,0,0,1,BRESCH Nathan,9'31'',6.0,MIM,Ibal,067,G-E,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,14:0020052008100199,67,G-E,Pas de label,2019,cross
1,1,1,2,BOYER Elliot,9'35'',5.0,MIM,Unitas Brumath,067,G-E,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,14:0020052008100199,67,G-E,Pas de label,2019,cross
2,2,2,3,GOETTEN Moritz (Ger),9'41'',7.0,BEM,Asv Landau,,,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,14:0020052008100199,67,G-E,Pas de label,2019,cross
3,3,3,4,LECHNER Arnaud,9'49'',7.0,BEM,Unitas Brumath,067,G-E,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,14:0020052008100199,67,G-E,Pas de label,2019,cross
4,4,4,5,BOSSUYT Thibault,9'56'',6.0,MIM,Ana Sg Wantzenau,067,G-E,C3-BE-MI,CROSS DE NOEL,WISSEMBOURG,22/12/19,14:0020052008100199,67,G-E,Pas de label,2019,cross


### Nettoyage base / retraitement données

In [3]:
#Suppression colonnes inutiles

data_tot.drop(data_tot.columns[[0,1,14]], axis=1, inplace=True)

# Gestion valeurs manquantes + retraitement variable année de naissance

data_tot["Annee_naissance"]=data_tot["Annee_naissance"].fillna(9999)
data_tot["Annee_naissance"]=data_tot["Annee_naissance"].astype(int)
data_tot["Annee_naissance"]=data_tot["Annee_naissance"].apply(lambda x: x + 2000 if 0 <= x <= 23 else (x + 1900 if 24 <= x <= 99 else x))

# Suppression des lignes pour lesquelles la valeur de la performance est manquante ou non valide

data_tot["Performance"]=data_tot["Performance"].apply(lambda x: np.nan if x == '-' else x)
data_tot.dropna(subset=["Performance"], inplace=True)

# On supprime les valeurs qi des Performances concernées pour ne garder uniquement que la valeur de la performance 

data_tot['Performance']=data_tot['Performance'].str.replace('qi', '')
data_tot['Performance']=data_tot['Performance'].str.replace('q', '')
data_tot['Performance']=data_tot['Performance'].str.replace('Q', '')

# On garde uniquement les valeurs entre parenthèses pour les performances possédant 2 valeurs 

def extraire_temps_parentheses(temps):
    match = re.search(r'\(([^)]+)\)', temps)
    if match:
        return match.group(1)
    else:
        return temps
    
data_tot['Performance'] = data_tot['Performance'].apply(extraire_temps_parentheses)


# Tri du dataframe

data_tot_sorted=data_tot.sort_values(by=['Nom_athlete', 'Annee_naissance', 'Date_competition'])

# Création et application de la fonction permettant d'obtenir le détail de la catégorie Master 

def cat_master(row): 
    if 35 <= row["Annee_competition"]-row["Annee_naissance"] <= 39: 
        return "M0"
    elif 40 <= row["Annee_competition"]-row["Annee_naissance"] <= 44: 
        return "M1"
    elif 45 <= row["Annee_competition"]-row["Annee_naissance"] <= 49: 
        return "M2"
    elif 50 <= row["Annee_competition"]-row["Annee_naissance"] <= 54: 
        return "M3"
    elif 55 <= row["Annee_competition"]-row["Annee_naissance"] <= 59: 
        return "M4"
    elif 60 <= row["Annee_competition"]-row["Annee_naissance"] <= 64: 
        return "M5"
    elif 65 <= row["Annee_competition"]-row["Annee_naissance"] <= 69: 
        return "M6"
    elif 70 <= row["Annee_competition"]-row["Annee_naissance"] <= 74: 
        return "M7"
    elif 75 <= row["Annee_competition"]-row["Annee_naissance"] <= 79: 
        return "M8"
    elif 80 <= row["Annee_competition"]-row["Annee_naissance"] <= 84: 
        return "M9"
    elif 85 <= row["Annee_competition"]-row["Annee_naissance"]: 
        return "M10"
    else:
        return row['Categorie']
    
data_tot_sorted['Categorie_precise']=data_tot_sorted.apply(lambda x: cat_master(x), axis=1)

# Jointure du nom et de l'année de naissance afin d'éviter les doublons de nom 

data_tot_sorted['Nom2'] = pd.concat([data_tot_sorted['Nom_athlete'], data_tot_sorted['Annee_naissance'].astype(str)], axis=1).agg('-'.join, axis=1) #agg permet d'appliquer la fonction join à chaque ligne

# Création de la variable contenant la distance parcourue lors de l'épreuve

def extract_distance(row): 
    
    if row['Type_competition'] in ('stade', 'salle') and '0m' in (row['Epreuve']).lower():
        return row['Epreuve'].split('/')[0]
    elif row['Type_competition'] == 'cross':
        return 'Cross'
    elif row['Type_competition'] == 'HS' and 'marathon' in (row['Epreuve']).lower():
        return '42,195km'
    elif row['Type_competition'] == 'HS' and ('1/2 marathon' in (row['Epreuve']).lower() or 'semi marathon' in (row['Epreuve']).lower() or 'semi-marathon' in (row['Epreuve']).lower()):
        return '21,097km'
    elif row['Type_competition'] == 'HS' and 'km' in (row['Epreuve']).lower():
        return re.sub(r'[^\d,.]', '', row['Epreuve']) + 'km'
    else:
        return 'Autres'
    
data_tot_sorted['Distance'] = data_tot_sorted.apply(lambda x: extract_distance(x), axis=1)

# Création variable nombre autres et indicateur si autres ou non 

count_autres = data_tot_sorted.groupby('Nom2')['Distance'].apply(lambda x: (x == 'Autres').sum()).reset_index(name='Nombre_Autres')
data_tot_sorted = pd.merge(data_tot_sorted, count_autres, on='Nom2', how='left')
data_tot_sorted.head()

# Création de la variable précisant le genre de l'individu 

data_tot_sorted['Categorie']=data_tot_sorted['Categorie'].astype(str)
data_tot_sorted['Genre'] = data_tot_sorted['Categorie'].apply(lambda x: x[-1])


#### Création de la table regroupant les meilleures performances par athlète pour chaque distance

In [4]:
meilleures_performances = data_tot_sorted.groupby(['Nom2', 'Distance'])['Performance'].agg(min).reset_index()
meilleures_performances.rename(columns={'Performance':'Meilleure_Performance'}, inplace=True)
meilleures_performances.head()

C:\Users\alexa\AppData\Local\Temp\ipykernel_84248\2017267375.py:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  meilleures_performances = data_tot_sorted.groupby(['Nom2', 'Distance'])['Performance'].agg(min).reset_index()


In [4]:
# Création du sous tableau nous intéressant 

data_genre = data_tot_sorted[~data_tot_sorted["Distance"].isin(['Autres', 'Cross'])]

def convertir_format_performance(performance):
    
    if re.match(r'(\d+):(\d+):(\d+)', performance) or re.match(r'(\d+):(\d+)', performance) or re.match(r'(\d+):(\d+)', performance):
        heures, minutes, secondes, centiemes = 0, 0, 0, 0

        # Chercher le format hh:mm:ss
        match = re.match(r'(\d+):(\d+):(\d+)', performance)
        if match:
            heures, minutes, secondes = map(int, match.groups())

        # Chercher le format hh:mm
        elif re.match(r'(\d+):(\d+)', performance):
            heures, minutes = map(int, performance.split(':'))

        # Chercher le format mm:ss
        elif re.match(r'(\d+):(\d+)', performance):
            minutes, secondes = map(int, performance.split(':'))

        # Construire le format de sortie
        format_sortie = ""
        if heures > 0:
            format_sortie += f"{heures}h"
        if minutes > 0:
            format_sortie += f"{minutes}'"
        if secondes > 0:
            format_sortie += f"{secondes}\""
        if centiemes > 0:
            format_sortie += f"{centiemes:02}"
    else:
        format_sortie=performance

    return format_sortie

data_genre['Performance2']=data_genre['Performance'].apply(convertir_format_performance)

# Fonction permettant de convertir les performances en nombre de secondes

def convertir_en_secondes(temps):
    heures, minutes, secondes = 0, 0, 0
    temps_match = re.match(r'(?:(\d+)h)?(?:(\d+)\'?)?(?:(\d+)\'\'([\d\.]+)?)?', temps)
    
    if temps_match:
        heures = int(temps_match.group(1)) if temps_match.group(1) else 0
        minutes = int(temps_match.group(2)) if temps_match.group(2) else 0
        secondes = int(temps_match.group(3)) if temps_match.group(3) else 0
        millisecondes = float(temps_match.group(4)) if temps_match.group(4) else 0

    temps_en_secondes = heures * 3600 + minutes * 60 + secondes + millisecondes
    
    return temps_en_secondes

data_genre["Performance2"] = data_genre["Performance2"].apply(convertir_en_secondes)


C:\Users\alexa\AppData\Local\Temp\ipykernel_81500\1222247194.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_genre['Performance2']=data_genre['Performance'].apply(convertir_format_performance)


In [12]:


# On veut pour chaque distance le temps moyen / médian et le meilleur temps par genre 

perfs_par_genre = data_genre.groupby(['Distance', 'Genre'])['Performance2'].agg(['mean', 'median', 'min']).reset_index()

# On reconvertit les temps dans leur format d'origine

def convertir_en_format_temps(total_secondes):
    heures, reste = divmod(total_secondes, 3600)
    minutes, secondes = divmod(reste, 60)
    return f"{int(heures)}h{int(minutes)}'{int(secondes)}\"{int((secondes % 1) * 100)}"


#perfs_par_genre.rename(columns={'Performance':'Temps_moyen'}, inplace=True)
perfs_par_genre.head()

,Distance,Genre,mean,median,min
0,",1,3km",F,26160.000000,26160.0,26160.0
1,",1,3km",M,30300.000000,31560.0,21960.0
2,",15km",F,8545.350000,8723.0,7021.0
3,",15km",M,7575.888889,7616.0,6622.0
4,",5km",M,1465.000000,1465.0,1465.0


C:\Users\alexa\AppData\Local\Temp\ipykernel_81500\2010001321.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_genre["Performance2"] = data_genre["Performance2"].apply(convertir_en_secondes)
